# Load data

## Imports

In [1]:
# Імпорт необхідних бібліотек
import os
import urllib.request
import tarfile

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

2024-06-24 00:24:06.393453: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 00:24:07.214840: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 00:24:10.031265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 00:24:16.942538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load and prepare data

In [ ]:
# Функція для завантаження та розпакування CIFAR-10 датасету
def download_and_extract_cifar10(destination_folder="data"):
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = os.path.join(destination_folder, "cifar-10-python.tar.gz")

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    if not os.path.exists(filename):
        print("Downloading CIFAR-10 dataset...")
        urllib.request.urlretrieve(url, filename)
        print("Download complete.")

    with tarfile.open(filename, "r:gz") as tar:
        print("Extracting CIFAR-10 dataset...")
        tar.extractall(path=destination_folder)
        print("Extraction complete.")

# Виклик функції для завантаження та розпакування датасету
download_and_extract_cifar10()

# Завантаження CIFAR-10 датасету через TensorFlow
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Перетворення міток у one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Нормалізація даних
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Виведення форми масивів даних
print("Training data shape: ", x_train.shape)
print("Training labels shape: ", y_train.shape)
print("Test data shape: ", x_test.shape)
print("Test labels shape: ", y_test.shape)

## See data

In [ ]:
# Візуалізація деяких зображень з тренувального набору
def plot_sample_images(x, y, classes, num_samples=10):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 15))
    axes = axes.flatten()
    for i in range(num_samples):
        axes[i].imshow(x[i])
        axes[i].set_title(classes[np.argmax(y[i])])
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

# Назви класів CIFAR-10
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Візуалізація перших 10 зображень з тренувального набору
plot_sample_images(x_train, y_train, class_names)